# データインポート

In [15]:
import numpy as np
import pandas as pd
import os
import sys
from datetime import datetime, timedelta

sys.path.append("../")
from backend.elastic_manager.elastic_manager import ElasticManager

from elasticsearch import Elasticsearch, helpers

# ElasticManager.es = Elasticsearch(hosts='http://doc-elastic:9200', http_auth=("elastic", "P@ssw0rd"), timeout=50000)
# ElasticManager.es

## 概要確認

In [16]:
# 全ショットデータ
df_angle = pd.read_csv("20210810/0to1K/log0/log000_角度とR2107311654.csv", encoding="shift-jis")
df_angle.head(5)

,DATE,TIME,SHOT数,角度,R半径,板厚SHOT,板厚,板形状SHOT,ピークtoボトム,ピーク距離,ボトム距離,ピークボトム距離
0,2021/07/31,16:54:25,1,15.0283,71.8934,20,1.406,21,5.7352,87.900,194.625,108.975
1,2021/07/31,16:54:27,2,15.5637,74.3223,21,1.466,22,5.8597,73.575,194.625,123.150
2,2021/07/31,16:54:29,3,17.0261,74.7403,22,1.477,23,5.8780,88.875,194.250,108.075
3,2021/07/31,16:54:31,4,18.2962,73.1533,23,1.428,24,5.3836,80.025,194.625,116.325
4,2021/07/31,16:54:33,5,17.0462,71.8284,24,1.435,25,5.1383,87.150,194.625,109.800


In [17]:
# 1ショット分のデータ
df_stroke = pd.read_csv("20210810/0to1K/log1/log000_ストロークと歪210731165425.csv", encoding="shift-jis")
df_stroke.head(10)

,DATE,TIME,INTERVAL,SHOT数,歪量,ストローク
0,2021/07/31,16:54:25,0,9,-7,19.79
1,2021/07/31,16:54:25,99,9,-7,19.79
2,2021/07/31,16:54:25,102,9,-6,19.76
3,2021/07/31,16:54:25,98,9,-6,19.76
4,2021/07/31,16:54:25,107,9,-5,19.75
5,2021/07/31,16:54:25,93,9,-5,19.75
6,2021/07/31,16:54:25,104,9,-6,19.77
7,2021/07/31,16:54:25,96,9,-6,19.77
8,2021/07/31,16:54:25,105,9,4,19.77
9,2021/07/31,16:54:25,95,9,4,19.77


In [18]:
ElasticManager.delete_index("j-20210731165425-data")
ElasticManager.delete_index("j-20210731165425-meta")

ElasticManager.create_index("j-20210731165425-data")
ElasticManager.create_index("j-20210731165425-meta")

[2021/10/18 14:20:11.985] INFO - elastic_manager.py#delete_index:100: j-20210731165425-data deleted.
[2021/10/18 14:20:12.098] INFO - elastic_manager.py#delete_index:100: j-20210731165425-meta deleted.
[2021/10/18 14:20:12.527] INFO - elastic_manager.py#create_index:217: create index 'j-20210731165425-data' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'j-20210731165425-data'}
[2021/10/18 14:20:12.933] INFO - elastic_manager.py#create_index:217: create index 'j-20210731165425-meta' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'j-20210731165425-meta'}


True

## ショット単位データ

### 日時列追加

In [19]:
df_angle["timestamp"] = df_angle["DATE"] + ' ' + df_angle["TIME"]
df_angle["timestamp"] = df_angle["timestamp"].map(lambda x: datetime.strptime(x, "%Y/%m/%d %H:%M:%S") - timedelta(hours=9))
df_angle.drop(columns=["DATE", "TIME"], inplace=True)

In [20]:
dict_angle = df_angle.to_dict(orient="records")
dict_angle

[{'SHOT数': 1,
  '角度': 15.0283,
  'R半径': 71.8934,
  '板厚SHOT': 20,
  '板厚': 1.406,
  '板形状SHOT': 21,
  'ピークtoボトム': 5.7352,
  'ピーク距離': 87.9,
  'ボトム距離': 194.625,
  'ピークボトム距離': 108.975,
  'timestamp': Timestamp('2021-07-31 07:54:25')},
 {'SHOT数': 2,
  '角度': 15.5637,
  'R半径': 74.3223,
  '板厚SHOT': 21,
  '板厚': 1.466,
  '板形状SHOT': 22,
  'ピークtoボトム': 5.8597,
  'ピーク距離': 73.575,
  'ボトム距離': 194.625,
  'ピークボトム距離': 123.15,
  'timestamp': Timestamp('2021-07-31 07:54:27')},
 {'SHOT数': 3,
  '角度': 17.0261,
  'R半径': 74.7403,
  '板厚SHOT': 22,
  '板厚': 1.477,
  '板形状SHOT': 23,
  'ピークtoボトム': 5.878,
  'ピーク距離': 88.875,
  'ボトム距離': 194.25,
  'ピークボトム距離': 108.075,
  'timestamp': Timestamp('2021-07-31 07:54:29')},
 {'SHOT数': 4,
  '角度': 18.2962,
  'R半径': 73.1533,
  '板厚SHOT': 23,
  '板厚': 1.428,
  '板形状SHOT': 24,
  'ピークtoボトム': 5.3836,
  'ピーク距離': 80.025,
  'ボトム距離': 194.625,
  'ピークボトム距離': 116.325,
  'timestamp': Timestamp('2021-07-31 07:54:31')},
 {'SHOT数': 5,
  '角度': 17.0462,
  'R半径': 71.8284,
  '板厚SHOT': 24,
  '板厚': 1.435,
 

In [21]:
actions = (
    {"_index": "j-20210731165425-meta", "_source": x} for x in dict_angle
)
helpers.bulk(ElasticManager.es, actions, chunk_size=5000, stats_only=True, raise_on_error=False)

(1000, 0)

## 角度外れ値除去

In [22]:
df_angle[df_angle["角度"] < 0]

,SHOT数,角度,R半径,板厚SHOT,板厚,板形状SHOT,ピークtoボトム,ピーク距離,ボトム距離,ピークボトム距離,timestamp
914,915,-100000.0,73.1096,934,1.512,935,4.2556,69.6,194.25,124.65,2021-07-31 08:28:23


In [23]:
df_angle_without_outlire = df_angle.drop(914)

In [24]:
df_angle_without_outlire[df_angle_without_outlire["角度"] < 0]

,SHOT数,角度,R半径,板厚SHOT,板厚,板形状SHOT,ピークtoボトム,ピーク距離,ボトム距離,ピークボトム距離,timestamp


### 再インポート

In [25]:
ElasticManager.delete_index("j-20210731165425-meta")
ElasticManager.create_index("j-20210731165425-meta")

[2021/10/18 14:20:16.679] INFO - elastic_manager.py#delete_index:100: j-20210731165425-meta deleted.
[2021/10/18 14:20:17.172] INFO - elastic_manager.py#create_index:217: create index 'j-20210731165425-meta' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'j-20210731165425-meta'}


True

In [26]:
dict_angle_without_outlire = df_angle_without_outlire.to_dict(orient="records")
dict_angle_without_outlire

[{'SHOT数': 1,
  '角度': 15.0283,
  'R半径': 71.8934,
  '板厚SHOT': 20,
  '板厚': 1.406,
  '板形状SHOT': 21,
  'ピークtoボトム': 5.7352,
  'ピーク距離': 87.9,
  'ボトム距離': 194.625,
  'ピークボトム距離': 108.975,
  'timestamp': Timestamp('2021-07-31 07:54:25')},
 {'SHOT数': 2,
  '角度': 15.5637,
  'R半径': 74.3223,
  '板厚SHOT': 21,
  '板厚': 1.466,
  '板形状SHOT': 22,
  'ピークtoボトム': 5.8597,
  'ピーク距離': 73.575,
  'ボトム距離': 194.625,
  'ピークボトム距離': 123.15,
  'timestamp': Timestamp('2021-07-31 07:54:27')},
 {'SHOT数': 3,
  '角度': 17.0261,
  'R半径': 74.7403,
  '板厚SHOT': 22,
  '板厚': 1.477,
  '板形状SHOT': 23,
  'ピークtoボトム': 5.878,
  'ピーク距離': 88.875,
  'ボトム距離': 194.25,
  'ピークボトム距離': 108.075,
  'timestamp': Timestamp('2021-07-31 07:54:29')},
 {'SHOT数': 4,
  '角度': 18.2962,
  'R半径': 73.1533,
  '板厚SHOT': 23,
  '板厚': 1.428,
  '板形状SHOT': 24,
  'ピークtoボトム': 5.3836,
  'ピーク距離': 80.025,
  'ボトム距離': 194.625,
  'ピークボトム距離': 116.325,
  'timestamp': Timestamp('2021-07-31 07:54:31')},
 {'SHOT数': 5,
  '角度': 17.0462,
  'R半径': 71.8284,
  '板厚SHOT': 24,
  '板厚': 1.435,
 

In [27]:
actions = (
    {"_index": "j-20210731165425-meta", "_source": x} for x in dict_angle_without_outlire
)
helpers.bulk(ElasticManager.es, actions, chunk_size=5000, stats_only=True, raise_on_error=False)

(999, 0)

## ストローク（サンプル単位データ）

In [42]:
import glob

files = glob.glob("20210810/0to1K/log1/*")
files.sort()

sequential_number = 0

df_stroke = pd.DataFrame([])

for file in files:
    df = pd.read_csv(file, encoding="shift-jis")
    df["sequential_number"] = df.index + sequential_number
    sequential_number += len(df)
    
    df_stroke = pd.concat([df_stroke, df])

df_stroke.reset_index(inplace=True)
df_stroke.tail(5)

,index,DATE,TIME,INTERVAL,SHOT数,歪量,ストローク,sequential_number
2942127,2939,2021/07/31,17:31:21,99,1008,19,20.03,2942127
2942128,2940,2021/07/31,17:31:21,100,1008,19,20.03,2942128
2942129,2941,2021/07/31,17:31:21,99,1008,9,20.03,2942129
2942130,2942,2021/07/31,17:31:21,101,1008,9,20.01,2942130
2942131,2943,2021/07/31,17:31:21,99,1008,9,20.01,2942131


### 前処理

In [43]:
def concat_timestamp_and_interval(x):
    return x["timestamp"] + timedelta(microseconds=int(x["CUM_INTERVAL"]))

# 累積INTERVAL
df_stroke["CUM_INTERVAL"] = df_stroke["INTERVAL"].cumsum()
# timestamp列
df_stroke["timestamp"] = df_stroke["DATE"] + ' ' + df_stroke["TIME"]
df_stroke["timestamp"] = df_stroke["timestamp"].map(lambda x: datetime.strptime(x, "%Y/%m/%d %H:%M:%S") - timedelta(hours=9))
df_stroke["timestamp"] = df_stroke.apply(concat_timestamp_and_interval, axis=1)
# df_stroke["timestamp"] = df_stroke["timestamp"].map(lambda x: x.timestamp())

df_stroke.drop(columns=["DATE", "TIME", "INTERVAL", "SHOT数", "CUM_INTERVAL"], inplace=True)
df_stroke.rename(columns={"歪量":"load01", "ストローク":"stroke_displacement"}, inplace=True)

In [44]:
df_stroke.tail(100)

,index,load01,stroke_displacement,sequential_number,timestamp
2942032,2844,-17,18.40,2942032,2021-07-31 08:36:15.100826
2942033,2845,-17,18.40,2942033,2021-07-31 08:36:15.100921
2942034,2846,9,18.41,2942034,2021-07-31 08:36:15.101026
2942035,2847,9,18.41,2942035,2021-07-31 08:36:15.101121
2942036,2848,-4,18.56,2942036,2021-07-31 08:36:15.101225
...,...,...,...,...,...
2942127,2939,19,20.03,2942127,2021-07-31 08:36:15.110322
2942128,2940,19,20.03,2942128,2021-07-31 08:36:15.110422
2942129,2941,9,20.03,2942129,2021-07-31 08:36:15.110521
2942130,2942,9,20.01,2942130,2021-07-31 08:36:15.110622


### ELSに出力

In [45]:
dict_stroke = df_stroke.to_dict(orient="records")
# dict_stroke

In [46]:
actions = (
    {"_index": "j-20210731165425-data", "_source": x} for x in dict_stroke
)
helpers.bulk(ElasticManager.es, actions, chunk_size=5000, stats_only=True, raise_on_error=False)

(2942132, 0)

## 列名を変更してpkl出力

In [47]:
dir = "20210810/machine-j-20210731165425"

if not os.path.exists(dir):
    os.makedirs(dir)

In [48]:
import glob
import shutil

files = glob.glob("20210810/0to1K/log1/*")
files.sort()

def concat_timestamp_and_interval(x):
    return x["timestamp"] + timedelta(microseconds=int(x["CUM_INTERVAL"]))

sequential_number = 0

for file in files:
    df_stroke = pd.read_csv(file, encoding="shift-jis")

    # 累積INTERVAL
    df_stroke["CUM_INTERVAL"] = df_stroke["INTERVAL"].cumsum()
    # timestamp列
    df_stroke["timestamp"] = df_stroke["DATE"] + ' ' + df_stroke["TIME"]
    df_stroke["timestamp"] = df_stroke["timestamp"].map(lambda x: datetime.strptime(x, "%Y/%m/%d %H:%M:%S") - timedelta(hours=9))
    df_stroke["timestamp"] = df_stroke.apply(concat_timestamp_and_interval, axis=1)
    df_stroke["timestamp"] = df_stroke["timestamp"].map(lambda x: x.timestamp())
    
    # 連番
    df_stroke["sequential_number"] = df_stroke.index + sequential_number
    sequential_number += df_stroke.index[-1]

    df_stroke.drop(columns=["DATE", "TIME", "INTERVAL", "SHOT数", "CUM_INTERVAL"], inplace=True)
    df_stroke.rename(columns={"歪量":"load01", "ストローク":"stroke_displacement"}, inplace=True)
    
    pkl_file_name = os.path.splitext(os.path.basename(file))[0] + ".pkl"
    pkl_file_path = os.path.join(dir, pkl_file_name)
    
    # print(df_stroke.head())
    df_stroke.to_pickle(pkl_file_path)

In [49]:
df_stroke["timestamp"].map(lambda x: datetime.fromtimestamp(x))

0      2021-07-31 17:31:21.000000
1      2021-07-31 17:31:21.000095
2      2021-07-31 17:31:21.000195
3      2021-07-31 17:31:21.000295
4      2021-07-31 17:31:21.000395
                  ...            
2939   2021-07-31 17:31:21.293895
2940   2021-07-31 17:31:21.293995
2941   2021-07-31 17:31:21.294094
2942   2021-07-31 17:31:21.294195
2943   2021-07-31 17:31:21.294294
Name: timestamp, Length: 2944, dtype: datetime64[ns]

In [50]:
df_stroke

,load01,stroke_displacement,timestamp,sequential_number
0,-14,19.85,1.627720e+09,2938189
1,-14,19.85,1.627720e+09,2938190
2,-15,19.85,1.627720e+09,2938191
3,-15,19.85,1.627720e+09,2938192
4,2,19.85,1.627720e+09,2938193
...,...,...,...,...
2939,19,20.03,1.627720e+09,2941128
2940,19,20.03,1.627720e+09,2941129
2941,9,20.03,1.627720e+09,2941130
2942,9,20.01,1.627720e+09,2941131


In [79]:
datetime.fromtimestamp(df_stroke.timestamp[0])

datetime.datetime(2021, 7, 31, 8, 31, 21)

In [86]:
!ls 20210810/machine-j-20210731165425

log000_ストロークと歪210731165425.pkl  log500_ストロークと歪210731171307.pkl
log001_ストロークと歪210731165427.pkl  log501_ストロークと歪210731171309.pkl
log002_ストロークと歪210731165429.pkl  log502_ストロークと歪210731171311.pkl
log003_ストロークと歪210731165431.pkl  log503_ストロークと歪210731171313.pkl
log004_ストロークと歪210731165433.pkl  log504_ストロークと歪210731171315.pkl
log005_ストロークと歪210731165435.pkl  log505_ストロークと歪210731171317.pkl
log006_ストロークと歪210731165437.pkl  log506_ストロークと歪210731171319.pkl
log007_ストロークと歪210731165439.pkl  log507_ストロークと歪210731171321.pkl
log008_ストロークと歪210731165441.pkl  log508_ストロークと歪210731171324.pkl
log009_ストロークと歪210731165443.pkl  log509_ストロークと歪210731171326.pkl
log010_ストロークと歪210731165445.pkl  log510_ストロークと歪210731171328.pkl
log011_ストロークと歪210731165448.pkl  log511_ストロークと歪210731171330.pkl
log012_ストロークと歪210731165450.pkl  log512_ストロークと歪210731171332.pkl
log013_ストロークと歪210731165452.pkl  log513_ストロークと歪210731171334.pkl
log014_ストロークと歪210731165454.pkl  log514_ストロークと歪210731171336.pkl
log015_ストロークと歪210731165456.pkl  log515_ストロークと歪210731171

In [51]:
a = pd.read_pickle("20210810/machine-j-20210731165425/log000_ストロークと歪210731165425.pkl")

In [54]:
datetime.fromtimestamp(a.timestamp[0])

datetime.datetime(2021, 7, 31, 16, 54, 25)